In [2]:
import lmdb
import json
import openai
import numpy as np
import pandas as pd

In [ ]:
client = openai.OpenAI()

In [3]:
class VecterDB:
    DB_DIR = "."
    
    def __init__(self):
        self.env = lmdb.open(VecterDB.DB_DIR, map_size=10485760*4)
    
    @staticmethod
    def similarity(v1, v2):  # return dot product of two vectors
        return np.dot(v1, v2)
    
    @staticmethod
    def get_embedding(content, model='text-embedding-ada-002'):
        response = client.embeddings.create(input=content, model=model)
        vector = response.data[0].embedding
        return vector
    
    def make_vector_db(self, df):
        with self.env.begin(write=True) as txn:
            for id, [question, answer] in df[["question", "answer"]].iterrows():
                txn.put(f"Q_{id}".encode(), question.encode())
                txn.put(f"A_{id}".encode(), answer.encode())
                
                question = json.dumps(VecterDB.get_embedding(question))
                answer = json.dumps(VecterDB.get_embedding(answer))
                txn.put(f"QE_{id}".encode(), question.encode())
                txn.put(f"AE_{id}".encode(), answer.encode())
            
            txn.put(b"num_samples", str(len(df)).encode())
                
    def get_num(self):
        with self.env.begin() as txn:
            return int(txn.get(b'num_samples').decode()) 
            
    def get_qe(self, idx):
        with self.env.begin() as txn:
            question = json.loads(txn.get(f"QE_{idx}".encode()).decode())
            return question
        
    def get_ae(self, idx):
        with self.env.begin() as txn:
            question = json.loads(txn.get(f"AE_{idx}".encode()).decode())
            return question
        
    def get_q(self, idx):
        with self.env.begin() as txn:
            question = txn.get(f"Q_{idx}".encode()).decode()
            return question
        
    def get_a(self, idx):
        with self.env.begin() as txn:
            question = txn.get(f"A_{idx}".encode()).decode()
            return question
    
        
    def __del__(self):
        self.env.close()

    def get_pisible_answers(self, question, t_num):
        num = self.get_num()
        QE = self.get_embedding(question)
        answers = []
        for id in range(num):
            qe = self.get_qe(id)
            sim = self.similarity(QE, qe)
            answers.append([sim, id])
            answers.sort(reverse=True)
            if len(answers) > t_num:
                answers.pop()
        
        return [[self.get_a(id), sim] for sim, id in answers]

In [4]:
df = pd.read_excel("./../crawling/result.xlsx")
db = VecterDB()
# db.make_vector_db(df)

In [5]:
question = "자료2에 나와있는 공급량을 자료1에 그리면 수요곡선과 공급곡선의 가격이 50과 60사이에서 만나고 60에 더 가깝게 그려지지만 딱 60원을 지나지는 않는데 어떻게 60원이라고 할 수 있는건가요?"
# question = "근로소득,사업소득,이전소득,재산소득이먼지예를들어설명해주세요"
question = "수출 제품의 가격 경쟁률이 올라가면 순수출이 늘고 그럼 결국 총수요가 증가하는게 왜 아닌가요?왜 총공급이 증가하는건지 이해가 안됩니다ㅠㅠ"
db.get_pisible_answers(question, 7)

[["안녕하세요.질문에 답변드리겠습니다.'국내 기업들의 에너지 절감 노력으로 수출 제품의 가격 경쟁력이 향상되었다.'해당 내용의 경우 변동 원인인 '에너지 절감 노력'으로 총수요 혹은 총공급의 변화를 파악해야 합니다.에너지 절감 노력은 생산 기술 향상 또는 생산 비용의 감소로 파악해야 하므로 총공급 증가로 분석해야 합니다.총수요와 총공급의 변동과 관련해서는 '변동 원인'이 무엇인지 파악하는 것이 중요합니다.문제 풀이시에 '변동 원인'을 찾는 연습을 한다면 좀 더 쉽게 문제 해결을 할 수 있습니다.답변이 학습에 도움이 되었기를 바랍니다.",
  0.9999999677584885],
 ["안녕하세요.질문에 답변드리겠습니다.공급자 입장에서 미래에 가격이 상승할 것으로 예상이 된다면현재 공급을 하는 것보다 상승한 가격으로 공급할 수 있는 미래에 공급을 하는 것이 더 이득이 될 것입니다.결국 공급자는 미래에 공급할 것을 목적으로 '현재의 공급은 감소'시키게 되는 것입니다.공급 변동의 요인에서는 '미래 가격 예상'은 예상에 따라 '현재의 공급'을 어떻게 변화시킬 것인지를분석해야 하는 것입니다.답변이 학습에 도움이 되었기를 바랍니다.",
  0.9181551038451005],
 ['총수요, 총공급의 변동 파트에서…박봄 선생님께서 "국내 기업들이 에너지 절감 노력으로 수출 제품의 가격 경쟁력이 향상되었다." 이 부부분을 총공급 변동으로 설명하셨는데요. 수출 상품의 가격 경쟁력이 향상되면 수출이 증가할 것이고, 그럼 순수출이 증가해서 총수요가 증가하는 것이 맞지 않나요?위 질문에 관한 답변입니다.일단 질문에서 말씀하신 \'국내 기업들이 에너지 절감으로 수출 제품의 가격 경쟁력이 향상되었다.\'는 부분에서 에너지 절감 노력은 생산비의 절감이라고 볼 수 있고, 총공급을 늘리는 직접적인 영향에 해당됩니다. 물론, 그 이후에 수출경쟁력이 높아졌기 때문에 순수출은 증가할 것입니다.수요, 공급에서 학습하셨던 것을 기억해 보시면 비슷하다고 할 수 있습니다. 원인이냐 결과냐의 문제로 보시